## Dendritic Spine Shape Analysis

In [ ]:
import numpy as np
from spine_segmentation import load_tif
from notebook_widgets import show_3d_image


# load tif
image: np.ndarray = load_tif("image.tif")
show_3d_image(image)

In [ ]:
from notebook_widgets import interactive_binarization

binarization_widget = interactive_binarization(image)
display(binarization_widget)

In [ ]:
from spine_segmentation import get_surface_points
from CGAL.CGAL_Point_set_3 import Point_set_3

# extract binarization result
binary_image = binarization_widget.result

# calculate surface points
surface_points: Point_set_3 = get_surface_points(binary_image)

In [ ]:
from CGAL.CGAL_Polyhedron_3 import Polyhedron_3
from CGAL.CGAL_Poisson_surface_reconstruction import poisson_surface_reconstruction
from notebook_widgets import show_3d_mesh


# construct surface mesh
surface_poly = Polyhedron_3()
poisson_surface_reconstruction(surface_points, surface_poly)

# export surface mesh to .off file
surface_poly.write_to_file("output/surface_mesh.off")

# render surface mesh
show_3d_mesh(surface_poly)

In [ ]:
from CGAL.CGAL_Surface_mesh_skeletonization import surface_mesh_skeletonization
from CGAL.CGAL_Polygon_mesh_processing import Polylines
from spine_segmentation import build_graph, build_correspondence, build_reverse_correpondnce


# get surface skeleton
skeleton_polylines = Polylines()
correspondence_polylines = Polylines()
surface_mesh_skeletonization(surface_poly, skeleton_polylines, correspondence_polylines)

# convert to more performant data format 
skeleton_graph = build_graph(skeleton_polylines)
corr = build_correspondence(correspondence_polylines)
reverse_corr = build_reverse_correpondnce(correspondence_polylines)

In [ ]:
from notebook_widgets import interactive_segmentation


# perform segmentation 
segmentation_widget = interactive_segmentation(surface_poly, corr, reverse_corr, skeleton_graph)
display(segmentation_widget)

In [ ]:
from spine_segmentation import get_spine_meshes


# extract segmentation result
segmentation = segmentation_widget.result

# extract spine meshes
spine_meshes = get_spine_meshes(surface_poly, segmentation)

In [ ]:
# define calculated metrics

metric_names = ["Area", "Volume"]

In [ ]:
from spine_metrics import calculate_metrics


# calculate metrics for each spine
metrics = []
for spine_mesh in spine_meshes:
    metrics.append(calculate_metrics(spine_mesh, metric_names)) 

In [ ]:
from notebook_widgets import select_spines_widget


# manually select correct spines
selection_widget = select_spines_widget(spine_meshes, surface_poly, metrics)
display(selection_widget)

In [ ]:
from spine_segmentation import get_final_segmentation
from notebook_widgets import show_segmented_mesh


# extract selected spines
spine_selection = selection_widget.result
final_spines = []
for i in range(len(spine_selection)):
    if spine_selection[i]:
        final_spines.append(spine_meshes[i])

# export selected spine meshes to .off files
for i, spine_mesh in enumerate(final_spines):
    filename = f"output/spine_{i}.off"
    spine_mesh.write_to_file(filename)

# generate final segmentation from selected spines
final_segmentation = get_final_segmentation(final_spines)

# render final segmentation
show_segmented_mesh(surface_poly, final_segmentation)